In [97]:
import os
import pandas as pd
from fuzzywuzzy import fuzz
import regex as re
import sys

In [22]:
def replace_return(string):
    clean_string = re.sub(r"\n", "", string)
    clean_string = re.sub(r"nan", "", clean_string)
    clean_string = re.sub(r" ", "", clean_string)
    return clean_string

def score_response(response):
    if response > 95:
        return 1
    else:
        return 0

In [65]:
path = "/Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/scoring/test_data/immediate_recall/Nature/AN99SC170624_SNORE_recall_in_person_2024-06-17_15h08.51.056.csv"
df = pd.read_csv(path)
df_short = df[["Word1","Word2", "Response.text"]].copy()  # Create a copy of the DataFrame slice
# Ensure all values are treated as strings, replacing NaNs with empty strings
df_short["Word2"] = df_short["Word2"].astype(str).fillna("")
df_short = df_short[df_short["Word2"] != "nan"]
df_short["Response.text"] = df_short["Response.text"].astype(str).fillna("")
df_short["Response.text"] = df_short["Response.text"].apply(replace_return)

#calculate distance based on Leveinstein algorithm
df_short["distance"] = df_short.apply(lambda row: fuzz.token_sort_ratio(row["Response.text"], row["Word2"]), axis=1)

#Calculate score of the participant
df_short["scoring"] = df_short["distance"].apply(score_response)
result = df_short["scoring"].sum()

print(f"Result: {result}")
df_short[["Word1", "Word2", "Response.text", "distance"]].head(50)

Result: 23


,Word1,Word2,Response.text,distance
1,Hafer,Gras,Gras,100
2,Säugetier,Fledermaus,Fledermaus,100
3,Kolibri,Thunfisch,Thunfisch,100
4,Krokodil,Seeigel,Seeigel,100
5,Sprosse,Brombeere,Brombeere,100
6,Spore,Bulle,,0
7,Wolf,Buche,Buche,100
8,Languste,Getreide,,0
9,Knoblauch,Spargel,Spagel,92
10,Lorbeer,Walnuss,Walnuss,100


In [76]:
need_to_check = df_short[(df_short["distance"] < 100) & (df_short["distance"] > 80)]
need_to_check

,Word1,Word2,Response.text,distance,scoring
9,Knoblauch,Spargel,Spagel,92,0


In [86]:
need_to_check.iloc[0, need_to_check.columns.get_loc('Response.text')]

'Spagel'

In [101]:
need_to_check

,Word1,Word2,Response.text,distance,scoring
9,Knoblauch,Spargel,Spagel,92,0


In [100]:
if need_to_check.shape[0] > 0:
    print(f"Some words are almost the same. Please check if they are.\n "
          f"Is {need_to_check.iloc[0,need_to_check.columns.get_loc('Word2')]} close enough to {need_to_check.iloc[0, need_to_check.columns.get_loc('Response.text')]}?\n"
          f"if yes, press the y key if no press the n key.")
else:
    print(f"Result: {result}")
    sys.exit()
    
    
response = input("Press y or n to continue: ")

if response == "y":
    # set value to 1 if "y" is pressed
    word2_value = need_to_check.iloc[0, need_to_check.columns.get_loc('Word2')]
    df_short.loc[df_short["Word2"] == word2_value, "scoring"] = 1
    print("\nThanks for rating this row.")
elif response== "n":
    print("\nAll done.")
else:
    print("\nPlease press y or n to continue.")
    

Some words are almost the same. Please check if they are.
 Is Spargel close enough to Spagel?
if yes, press the y key if no press the n key.

Thanks for rating this row.


In [95]:
# Use .loc[] to avoid SettingWithCopyWarning
word2_value = need_to_check.iloc[0, need_to_check.columns.get_loc('Word2')]
df_short.loc[df_short["Word2"] == word2_value, "scoring"] = 1

df_short

,Word1,Word2,Response.text,distance,scoring
1,Hafer,Gras,Gras,100,1
2,Säugetier,Fledermaus,Fledermaus,100,1
3,Kolibri,Thunfisch,Thunfisch,100,1
4,Krokodil,Seeigel,Seeigel,100,1
5,Sprosse,Brombeere,Brombeere,100,1
6,Spore,Bulle,,0,0
7,Wolf,Buche,Buche,100,1
8,Languste,Getreide,,0,0
9,Knoblauch,Spargel,Spagel,92,1
10,Lorbeer,Walnuss,Walnuss,100,1


## Comparing fuzzy wuzzy functions

In [54]:
string1 = "Daune"
string2 = "Daunen"

ratio = fuzz.ratio(string1, string2)
partial_ratio = fuzz.partial_ratio(string1, string2)
token_sort_ratio = fuzz.token_sort_ratio(string1, string2)

print(f"Ratio: {ratio}")
print(f"Partial Ratio: {partial_ratio}")
print(f"Token Sort Ratio: {token_sort_ratio}")

Ratio: 91
Partial Ratio: 100
Token Sort Ratio: 91
